In [ ]:
import os

import numpy as np

import asdf


np.random.seed(42)

def print_file(fn):
    """
    A helper function to print out an ASDF file
    """
    with open(fn, "r", encoding="unicode_escape") as f:
        print(f.read())

## Exercise 1
Create an [AsdfFile](https://asdf.readthedocs.io/en/latest/api/asdf.AsdfFile.html#asdf.AsdfFile) instance and build a tree containing all of the above supported types.

In [ ]:
af = asdf.AsdfFile()
af.tree = {
    "my_dict": {"a": 1, "b": 2},
    "my_list": [1, 2, 3],
    "my_string": "abc",
    "my_int": 1,
    "my_float": 1.1,
    "my_none": None,
}
af.info()

## Exercise 2
Recreate (if necessary) your custom tree containing all of the supported types and write it to an ASDF file. Open the file in a text editor and view the contents.

In [ ]:
af.write_to("my_file.asdf")
print_file("my_file.asdf")

## Exercise 3
Open one of the ASDF files created above. What is the type of value stored with the "asdf_library" library in the tree?

In [ ]:
af = asdf.open("my_file.asdf")
print(type(af["asdf_library"]))

## Exercise 4
Generate an ASDF file with:
- an array ("array")
- a second reference to the same array ("array_reference")
- an additional different array ("other_array")

Examine the file contents. Open the file and examine "array" and "array_reference". Do they still refer to the same object? (hint: YAML supports anchors and aliases to have multiple references to the same object).

In [ ]:
af = asdf.AsdfFile()
af["array"] = np.arange(42)
af["array_reference"] = af["array"]
af["other_array"] = np.ones((2, 3, 4), dtype=np.uint8)
af.write_to("arrays.asdf")
print_file("arrays.asdf")

In [ ]:
af = asdf.open("arrays.asdf")
af["array"] is af["array_reference"]

## Exercise 5
Save an ASDF file with a large array and a small view of the array. Open this file and change the view contents. This will require disabling memory mapping by passing `memmap=False` to [asdf.open](https://asdf.readthedocs.io/en/latest/api/asdf.open.html#asdf.open). What happens to the large array?

In [ ]:
af = asdf.AsdfFile()
af["big"] = np.zeros((100, 100), dtype=np.uint16)
af["view"] = af["big"][:10, :10]
af.write_to("view.asdf")
print_file("view.asdf")

In [ ]:
af = asdf.open("view.asdf", memmap=False)
af["view"][:] = 1
print(af["big"][:])

## Exercise 6
Generate an ASDF file with:
- one array compressed with "zlib"
- a second array that is uncompressed

What happens if you read and then rewrite the file to a new filename?

In [ ]:
af = asdf.AsdfFile()
af["uncompressed"] = np.arange(42)
af["compressed"] = np.zeros(100)
af.set_array_compression(af["compressed"], "zlib")
af.write_to("compressed.asdf")
print_file("compressed.asdf")

In [ ]:
af = asdf.open("compressed.asdf")
print(af.get_array_compression(af["compressed"]))
af.write_to("compressed_copy.asdf")

In [ ]:
af = asdf.open("compressed_copy.asdf")
print(af.get_array_compression(af["compressed"]))

## Exercise 7
Write an ASDF file containing the following `astropy` objects:
1. [Quantity](https://docs.astropy.org/en/stable/units/quantity.html)
2. A [model](https://docs.astropy.org/en/stable/api/astropy.modeling.Model.html#astropy.modeling.Model)

   Hint: The [astropy.modeling](https://docs.astropy.org/en/stable/modeling/index.html) package provides a framework for representing models and performing model evaluation and fitting. Models are initialized using their parameters like in the following example for [Gaussian1D](https://docs.astropy.org/en/stable/api/astropy.modeling.functional_models.Gaussian1D.html#astropy.modeling.functional_models.Gaussian1D):
   ```
   from astropy.modeling import models
   gauss = models.Gaussian1D(amplitude=10, mean=3, stddev=1.2)
   ```
3. A [Time](https://docs.astropy.org/en/stable/time/index.html) object

    Hint: The [astropy.time](https://docs.astropy.org/en/stable/time/ref_api.html#module-astropy.time) package provides functionality for manipulating times and dates. To initialize it supply a string and a [format](https://docs.astropy.org/en/stable/time/index.html#id3), or supply a datetime object.
    
4. A [ICRS](https://docs.astropy.org/en/stable/api/astropy.coordinates.ICRS.html) coordinate object.

In [ ]:
import astropy.units as u
from astropy.modeling import models
from astropy import coordinates, modeling, time

af = asdf.AsdfFile()
af["quantity"] = u.Quantity([1, 2, 3], u.meter)
af["qaussian"] = models.Gaussian1D(amplitude=10, mean=3, stddev=1.2)
af["time"] = time.Time.now()
af["coordinate"] = coordinates.ICRS(ra=0 * u.deg, dec=0 * u.deg)
af.write_to("astropy_objects.asdf")
print_file("astropy_objects.asdf")

## Exercise 8

What happens if you set "sequence_id" to a string and call [validate](https://asdf.readthedocs.io/en/latest/api/asdf.AsdfFile.html#asdf.AsdfFile.validate)?

In [ ]:
schema_contents = """%YAML 1.1
---
id: "http://example.com/schemas/your-custom-schema"
$schema: "http://stsci.edu/schemas/yaml-schema/draft-01"

type: object
properties:
  image:
    description: My awesome image

  exposure_id:
    type: integer
    description: Numeric identifier for this image

required: [image, exposure_id]"""

# and write this out to a file so we can use it below
with open("image_schema-1.0.0.yaml", "w") as f:
    f.write(schema_contents)

af = asdf.AsdfFile(custom_schema="image_schema-1.0.0.yaml")
af["image"] = [1, 2, 3]
af["exposure_id"] = "not_a_number"
af.validate()